# 🎯 Smart Resume Parser using NER
## GenAI Project - Unit 1: Hugging Face Pipelines

**Goal**: Automatically extract Name, University, and Company from resume text using BERT-based Named Entity Recognition

**Tech Stack**:
- `pipeline('ner')` from Hugging Face
- `bert-base-NER` or `dslim/bert-base-NER`
- Python NLP libraries

In [1]:
from transformers import pipeline, AutoTokenizer, AutoModelForTokenClassification
import re
from collections import defaultdict
import pandas as pd

## 🤖 Step 3: Load BERT-based NER Pipeline

In [2]:
# Load the NER pipeline with a BERT-based model
# Using dslim/bert-base-NER - a fine-tuned BERT model for NER
ner_pipeline = pipeline(
    "ner",
    model="dslim/bert-base-NER",
    aggregation_strategy="simple"  # Groups tokens into complete entities
)

print("✅ NER Pipeline loaded successfully!")
print(f"Model: {ner_pipeline.model.name_or_path}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/829 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/433M [00:00<?, ?B/s]

Some weights of the model checkpoint at dslim/bert-base-NER were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/59.0 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


✅ NER Pipeline loaded successfully!
Model: dslim/bert-base-NER


## 📝 Step 4: Sample Resume Data

In [3]:
# Sample resume texts
sample_resumes = [
    """
    John Smith
    Email: john.smith@email.com | Phone: (555) 123-4567

    EDUCATION
    Master of Science in Computer Science
    Stanford University, 2020-2022
    Bachelor of Technology in Computer Engineering
    Massachusetts Institute of Technology, 2016-2020

    EXPERIENCE
    Senior Software Engineer
    Google LLC, Mountain View, CA | 2022-Present
    - Led development of machine learning pipelines
    - Worked with TensorFlow and PyTorch

    Software Development Intern
    Microsoft Corporation, Redmond, WA | Summer 2021
    - Developed Azure cloud solutions
    """,

    """
    Sarah Johnson
    sarah.j@gmail.com | LinkedIn: linkedin.com/in/sarahjohnson

    EDUCATION
    PhD in Artificial Intelligence
    University of California Berkeley, 2019-2023
    BS in Mathematics and Computer Science
    Carnegie Mellon University, 2015-2019

    WORK EXPERIENCE
    AI Research Scientist
    Meta (Facebook), Menlo Park, CA | 2023-Present
    - Research on large language models

    Machine Learning Engineer
    Amazon Web Services, Seattle, WA | 2021-2023
    - Built recommendation systems
    """,

    """
    Michael Chen
    michael.chen@outlook.com | (415) 987-6543

    EDUCATION
    MBA, Technology Management
    Harvard Business School, 2021-2023
    Bachelor of Science in Data Science
    University of Washington, 2017-2021

    PROFESSIONAL EXPERIENCE
    Product Manager - AI/ML
    Apple Inc., Cupertino, CA | 2023-Present
    - Managing Siri AI improvements

    Data Scientist
    Tesla Inc., Palo Alto, CA | 2021-2023
    - Worked on autonomous driving algorithms

    Research Assistant
    IBM Research, Yorktown Heights, NY | 2020-2021
    - Natural language processing research
    """
]

print(f"📄 Loaded {len(sample_resumes)} sample resumes")

📄 Loaded 3 sample resumes


## 🔍 Step 5: Define Resume Parser Function

In [4]:
def parse_resume(resume_text, ner_pipeline):
    """
    Extract Name, University, and Company from resume text using NER

    Args:
        resume_text: String containing resume content
        ner_pipeline: Hugging Face NER pipeline

    Returns:
        Dictionary with extracted entities
    """
    # Run NER on the resume text
    entities = ner_pipeline(resume_text)

    # Initialize result dictionary
    parsed_data = {
        'name': None,
        'universities': [],
        'companies': [],
        'all_persons': [],
        'all_organizations': []
    }

    # Common university keywords
    university_keywords = [
        'university', 'college', 'institute', 'school', 'mit', 'stanford',
        'harvard', 'berkeley', 'carnegie', 'caltech'
    ]

    # Common company identifiers
    company_identifiers = [
        'inc', 'llc', 'corporation', 'corp', 'ltd', 'company', 'co.',
        'google', 'microsoft', 'amazon', 'apple', 'meta', 'facebook',
        'tesla', 'ibm', 'intel', 'nvidia'
    ]

    # Process each entity
    for entity in entities:
        entity_text = entity['word'].strip()
        entity_type = entity['entity_group']

        # Extract PERSON entities (potential names)
        if entity_type == 'PER':
            parsed_data['all_persons'].append(entity_text)
            # First person mentioned is likely the candidate's name
            if parsed_data['name'] is None:
                parsed_data['name'] = entity_text

        # Extract ORGANIZATION entities
        elif entity_type == 'ORG':
            parsed_data['all_organizations'].append(entity_text)

            entity_lower = entity_text.lower()

            # Check if it's a university
            if any(keyword in entity_lower for keyword in university_keywords):
                if entity_text not in parsed_data['universities']:
                    parsed_data['universities'].append(entity_text)

            # Check if it's a company
            elif any(identifier in entity_lower for identifier in company_identifiers):
                if entity_text not in parsed_data['companies']:
                    parsed_data['companies'].append(entity_text)
            else:
                # Default to company if not clearly a university
                if entity_text not in parsed_data['companies']:
                    parsed_data['companies'].append(entity_text)

    return parsed_data

## 🚀 Step 6: Parse Sample Resumes

In [5]:
# Parse all sample resumes
results = []

for i, resume in enumerate(sample_resumes, 1):
    print(f"\n{'='*60}")
    print(f"📋 RESUME {i}")
    print(f"{'='*60}")

    parsed = parse_resume(resume, ner_pipeline)
    results.append(parsed)

    print(f"\n👤 Name: {parsed['name']}")
    print(f"\n🎓 Universities:")
    for uni in parsed['universities']:
        print(f"   • {uni}")

    print(f"\n🏢 Companies:")
    for company in parsed['companies']:
        print(f"   • {company}")

    print(f"\n📊 All Detected Entities:")
    print(f"   Persons: {parsed['all_persons']}")
    print(f"   Organizations: {parsed['all_organizations']}")


📋 RESUME 1

👤 Name: John Smith

🎓 Universities:
   • Stanford University
   • Massachusetts Institute of Technology

🏢 Companies:
   • Technology
   • Google LLC
   • Ten
   • ##sorFlow
   • PyTorch Software Development Inter
   • Microsoft Corporation

📊 All Detected Entities:
   Persons: ['John Smith']
   Organizations: ['Stanford University', 'Technology', 'Massachusetts Institute of Technology', 'Google LLC', 'Ten', '##sorFlow', 'PyTorch Software Development Inter', 'Microsoft Corporation']

📋 RESUME 2

👤 Name: Sarah Johnson

🎓 Universities:
   • Artificial Intelligence University of California Berkeley
   • Carnegie Mellon University

🏢 Companies:
   • Computer Science
   • ##P
   • Research Scientist
   • Facebook
   • Machine Learning Engineer Amazon Web Services

📊 All Detected Entities:
   Persons: ['Sarah Johnson']
   Organizations: ['Artificial Intelligence University of California Berkeley', 'Computer Science', 'Carnegie Mellon University', '##P', 'Research Scientist', 'Fa

## 📊 Step 7: Create Summary Table

In [6]:
# Create a summary DataFrame
summary_data = []

for i, result in enumerate(results, 1):
    summary_data.append({
        'Resume #': i,
        'Name': result['name'],
        'Universities': ', '.join(result['universities']),
        'Companies': ', '.join(result['companies']),
        'Total Orgs': len(result['all_organizations'])
    })

df_summary = pd.DataFrame(summary_data)
print("\n" + "="*80)
print("📈 RESUME PARSING SUMMARY")
print("="*80)
print(df_summary.to_string(index=False))


📈 RESUME PARSING SUMMARY
 Resume #          Name                                                                          Universities                                                                                                                  Companies  Total Orgs
        1    John Smith                            Stanford University, Massachusetts Institute of Technology                          Technology, Google LLC, Ten, ##sorFlow, PyTorch Software Development Inter, Microsoft Corporation           8
        2 Sarah Johnson Artificial Intelligence University of California Berkeley, Carnegie Mellon University                         Computer Science, ##P, Research Scientist, Facebook, Machine Learning Engineer Amazon Web Services           7
        3  Michael Chen                                                               Harvard Business School Technology Management, Science, Data Science, of Washington, AI, ML Apple Inc, Sir, Data Scientist Tesla Inc, IBM Research       

## 🎨 Step 8: Interactive Resume Parser (Your Own Resume!)

In [7]:
# Try with your own resume!
print("✍️ Paste your resume text below and run this cell:")
print("(Or modify the user_resume variable)\n")

user_resume = """
Paste your resume here...

Or try this example:

Emily Rodriguez
emily.rodriguez@email.com | (650) 555-0123

EDUCATION
Master of Computer Science
Georgia Institute of Technology, 2021-2023
Bachelor of Engineering in Software Engineering
University of Texas at Austin, 2017-2021

EXPERIENCE
Machine Learning Engineer
Netflix Inc., Los Gatos, CA | 2023-Present
- Developed recommendation algorithms

Software Engineering Intern
Salesforce, San Francisco, CA | Summer 2022
- Built CRM features using Python and React

Research Intern
NVIDIA Corporation, Santa Clara, CA | Summer 2021
- GPU optimization for deep learning
"""

# Parse the user's resume
if user_resume.strip() and "Paste your resume here" not in user_resume:
    print("\n🔍 Parsing your resume...\n")
    user_parsed = parse_resume(user_resume, ner_pipeline)

    print("="*60)
    print("📋 YOUR RESUME ANALYSIS")
    print("="*60)
    print(f"\n👤 Candidate Name: {user_parsed['name']}")
    print(f"\n🎓 Educational Institutions:")
    for uni in user_parsed['universities']:
        print(f"   ✓ {uni}")
    print(f"\n🏢 Work Experience:")
    for company in user_parsed['companies']:
        print(f"   ✓ {company}")
else:
    print("⚠️ Please paste your resume text in the 'user_resume' variable above!")

✍️ Paste your resume text below and run this cell:
(Or modify the user_resume variable)

⚠️ Please paste your resume text in the 'user_resume' variable above!


## 🧪 Step 9: Test NER with Direct Examples

In [8]:
# Test the raw NER pipeline output
test_sentence = "John Doe graduated from Harvard University and now works at Amazon Web Services."

print("Test Sentence:")
print(test_sentence)
print("\nNER Results:")

entities = ner_pipeline(test_sentence)
for entity in entities:
    print(f"  • {entity['word']:25} | Type: {entity['entity_group']:5} | Score: {entity['score']:.3f}")

Test Sentence:
John Doe graduated from Harvard University and now works at Amazon Web Services.

NER Results:
  • John Doe                  | Type: PER   | Score: 0.989
  • Harvard University        | Type: ORG   | Score: 0.994
  • Amazon Web Services       | Type: ORG   | Score: 0.999


## 💡 Step 10: Advanced Features - Entity Highlighting

In [9]:
def highlight_entities(text, ner_pipeline):
    """
    Highlight entities in the text with color coding
    """
    entities = ner_pipeline(text)

    # Create a color-coded version
    highlighted = text

    # Sort entities by start position (reverse to avoid index shifting)
    entities_sorted = sorted(entities, key=lambda x: x['start'], reverse=True)

    for entity in entities_sorted:
        start = entity['start']
        end = entity['end']
        entity_type = entity['entity_group']
        word = entity['word']

        # Create highlighted version
        if entity_type == 'PER':
            replacement = f"[PERSON: {word}]"
        elif entity_type == 'ORG':
            replacement = f"[ORG: {word}]"
        elif entity_type == 'LOC':
            replacement = f"[LOCATION: {word}]"
        else:
            replacement = f"[{entity_type}: {word}]"

        highlighted = highlighted[:start] + replacement + highlighted[end:]

    return highlighted

# Test with first resume
print("Original Resume (First 500 chars):")
print(sample_resumes[0][:500])
print("\n" + "="*80 + "\n")
print("Highlighted Entities:")
print(highlight_entities(sample_resumes[0][:500], ner_pipeline))

Original Resume (First 500 chars):

    John Smith
    Email: john.smith@email.com | Phone: (555) 123-4567

    EDUCATION
    Master of Science in Computer Science
    Stanford University, 2020-2022
    Bachelor of Technology in Computer Engineering
    Massachusetts Institute of Technology, 2016-2020

    EXPERIENCE
    Senior Software Engineer
    Google LLC, Mountain View, CA | 2022-Present
    - Led development of machine learning pipelines
    - Worked with TensorFlow and PyTorch

    Software Development Intern
    Microsof


Highlighted Entities:

    [PERSON: John Smith]
    Email: john.smith@email.com | Phone: (555) 123-4567

    EDUCATION
    Master of [MISC: Science] in [MISC: Computer Science]
    [ORG: Stanford University], 2020-2022
    Bachelor of [ORG: Technology] in [MISC: Computer Engineering]
    [ORG: Massachusetts Institute of Technology], 2016-2020

    EXPERIENCE
    Senior Software Engineer
    [ORG: Google LLC], [LOCATION: Mountain View], [LOCATION: CA] | 2022-

## 📈 Step 11: Statistics and Insights

In [10]:
# Calculate statistics across all resumes
total_universities = sum(len(r['universities']) for r in results)
total_companies = sum(len(r['companies']) for r in results)
total_orgs = sum(len(r['all_organizations']) for r in results)

print("📊 Overall Statistics:")
print(f"  • Total Resumes Processed: {len(results)}")
print(f"  • Total Universities Found: {total_universities}")
print(f"  • Total Companies Found: {total_companies}")
print(f"  • Total Organizations: {total_orgs}")
print(f"  • Avg Universities per Resume: {total_universities/len(results):.1f}")
print(f"  • Avg Companies per Resume: {total_companies/len(results):.1f}")

# Most common companies
all_companies = []
for r in results:
    all_companies.extend(r['companies'])

from collections import Counter
company_counts = Counter(all_companies)

print("\n🏆 Top Companies Mentioned:")
for company, count in company_counts.most_common(5):
    print(f"  {count}x {company}")

📊 Overall Statistics:
  • Total Resumes Processed: 3
  • Total Universities Found: 5
  • Total Companies Found: 20
  • Total Organizations: 26
  • Avg Universities per Resume: 1.7
  • Avg Companies per Resume: 6.7

🏆 Top Companies Mentioned:
  1x Technology
  1x Google LLC
  1x Ten
  1x ##sorFlow
  1x PyTorch Software Development Inter


## 🎯 Conclusion and Next Steps

### What We Built:
1. ✅ Loaded BERT-based NER model using `pipeline('ner')`
2. ✅ Extracted **Name** (PER entities)
3. ✅ Extracted **Universities** (ORG entities with university keywords)
4. ✅ Extracted **Companies** (ORG entities with company identifiers)
5. ✅ Created an interactive resume parser

### Enhancement Ideas:
- 🚀 Add email and phone extraction using regex
- 🚀 Extract skills using keyword matching
- 🚀 Add sentiment analysis for work descriptions
- 🚀 Use Question Answering to extract specific fields
- 🚀 Add PDF resume upload capability
- 🚀 Fine-tune BERT on resume-specific data

### Unit 1 Concepts Covered:
- ✅ Hugging Face Pipelines
- ✅ Named Entity Recognition (NER)
- ✅ BERT-based models
- ✅ Text preprocessing and entity extraction